In [13]:
import pandas as pd
import numpy as np

In [14]:
from sklearn.preprocessing import LabelEncoder


def preprocess(train,test,delete_columns = ['Name', 'PassengerId', 'SibSp', 'Parch', 'Ticket'],categorical_features=["Sex","Embarked","Cabin","Pclass"],to_categorical=True,use_cabin_prefix=True):
    data = pd.concat([train, test])
    data['Sex'].replace(['male','female'], [0, 1],inplace=True)
    data['Embarked'].fillna(('S'), inplace=True)
    data['Embarked'] = data['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    
    data["Age"].fillna(np.mean(data["Age"]),inplace=True)
    data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
    
    data["Cabin"].fillna("NaN",inplace=True) # Nullは変換しておかないとlabel encodingできない
    
    if use_cabin_prefix:
        data["Cabin"] = data["Cabin"].apply(lambda x: x[0]) # 部屋の先頭文字が大事かも？
    le = LabelEncoder()
    data["Cabin"] = le.fit_transform(data['Cabin'].values)
    if to_categorical:
        for feature in categorical_features:
            data[feature]=data[feature].astype("category")

    data.drop(delete_columns, axis=1, inplace=True)
    return data

In [15]:
def kesson_table(df): 
    null_val = df.isnull().sum() # series
    percent = 100 * df.isnull().sum()/len(df) #series
    kesson_table = pd.concat([null_val, percent], axis=1)
    kesson_table_ren_columns = kesson_table.rename(
    columns = {0 : '欠損数', 1 : '%'})
    return kesson_table_ren_columns

In [16]:
train=pd.read_csv("data/train.csv")
test=pd.read_csv("data/train.csv")
data=preprocess(train,test,to_categorical=False)

In [17]:
kesson_table(data)

,欠損数,%
Survived,0,0.0
Pclass,0,0.0
Sex,0,0.0
Age,0,0.0
Fare,0,0.0
Cabin,0,0.0
Embarked,0,0.0


In [18]:
data.head()

,Survived,Pclass,Sex,Age,Fare,Cabin,Embarked
0,0,3,0,22.0,7.2500,7,0
1,1,1,1,38.0,71.2833,2,1
2,1,3,1,26.0,7.9250,7,0
3,1,1,1,35.0,53.1000,2,0
4,0,3,0,35.0,8.0500,7,0


In [19]:
data.dtypes

Survived      int64
Pclass        int64
Sex           int64
Age         float64
Fare        float64
Cabin         int64
Embarked      int64
dtype: object

In [20]:
from sklearn.model_selection import train_test_split
train = data[:len(train)]
test = data[len(train):]

y_train_val = train['Survived']
X_train_val = train.drop('Survived', axis=1)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, random_state=0)

X_test = test.drop('Survived', axis=1)

In [21]:
import lightgbm as lgb

/Users/hikaru/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [22]:
gbm = lgb.LGBMClassifier(objective='binary')

# trainとvalidを指定し学習
gbm.fit(X_train, y_train, eval_set = [(X_val, y_val)],
        early_stopping_rounds=20,  # 20回連続でlossが下がらなかったら終了
        verbose=10,  # 10round毎に、lossを表示
)

Training until validation scores don't improve for 20 rounds
[10]	valid_0's binary_logloss: 0.441726
[20]	valid_0's binary_logloss: 0.391613
[30]	valid_0's binary_logloss: 0.373391
[40]	valid_0's binary_logloss: 0.358313
[50]	valid_0's binary_logloss: 0.367145
[60]	valid_0's binary_logloss: 0.380249
Early stopping, best iteration is:
[41]	valid_0's binary_logloss: 0.357592


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective='binary',
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [23]:
y_pred_val = gbm.predict(X_val, num_iteration=gbm.best_iteration_)

/Users/hikaru/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [29]:
# valの評価
from sklearn.metrics import accuracy_score
accuracy_score(y_val, y_pred_val)

0.8715083798882681